In [1]:
import uproot
from physlite_experiments import tree_arrays
import awkward as ak
import numpy as np
import time
import os

# Read branches

In [2]:
def filter_branch(branch):
        k = branch.name

        if not "Aux" in k:
            return False

        # the following don't contain data (in split files)
        if k.endswith("."):
            return False
        if "SG::" in k:
            return False
        if k.endswith("Base"):
            return False

        # are often empty
        # see https://github.com/scikit-hep/uproot4/issues/126
        # -> now fixed, but my custom deserialization does not work yet with them
        if "DescrTags" in k:
            return False
        
        return True

In [3]:
%%time
# file from user.nihartma.physlite_test_ttbar_split99.001_EXT0
f = uproot.open("user.nihartma.22884623.EXT0._000001.DAOD_PHYSLITE.test.pool.root")
tree = f["CollectionTree"]
array_dict = tree_arrays(tree, filter_branch=filter_branch)

CPU times: user 6.78 s, sys: 955 ms, total: 7.73 s
Wall time: 6.75 s


In [4]:
len(array_dict)

1121

What did we miss (assuming fully split Aux branches)?

In [5]:
all_aux = [
    k.split("/")[-1] for k in tree.keys("/(.*Aux\..+|.*AuxDyn\..+)/i")
    if not "xAOD::" in k
    and len(tree[k].branches) == 0
]
set(all_aux).difference(array_dict.keys())

{'EventInfoAux.detDescrTags.first', 'EventInfoAux.detDescrTags.second'}

# Build an nicer structure

We can also structure this much nicer and remove duplicated indices (e.g. all electron properties share the same offsets) - the naming conventions help us:

In [6]:
def regroup(array_dict):
    regrouped = {}
    for k_top in set(k.split(".")[0] for k in array_dict):
        if k_top == "EventInfoAux":
            # skip that for now - let's use EventInfoAuxDyn
            continue
        if k_top == "EventInfoAuxDyn":
            k_top = "EventInfoAux"
        # zip will put together jagged arrays with common offsets
        def ak_zip(depth_limit=2):
            return ak.zip(
                {k.replace(k_top, "")[1:] : array_dict[k] for k in array_dict if k_top in k},
                depth_limit=depth_limit
            )
        # for some containers this will work 2 levels, for some only up to 1
        try:
            v = ak_zip(depth_limit=2)
        except ValueError:
            v = ak_zip(depth_limit=1)
        regrouped[k_top.replace("AuxDyn", "").replace("Aux", "")] = v
    # lets restructure such that we get TrigMatchedObjets.<trigger-name>
    # instead of AnalysisHLT_<trigger_name>.TrigMatchedObjects
    trig_matched_objects = ak.zip(
        {
            k.replace("AnalysisHLT_", "") : regrouped[k].TrigMatchedObjects
            for k in regrouped if "AnalysisHLT" in k
        },
        depth_limit=1
    )
    for k in list(regrouped.keys()):
        if "AnalysisHLT" in k:
            regrouped.pop(k)
    regrouped["TrigMatchedObjects"] = trig_matched_objects
    return ak.zip(regrouped, depth_limit=1)

In [7]:
Events = regroup(array_dict)

In [8]:
Events.AnalysisElectrons.pt

<Array [[], [3.37e+03], ... [6.27e+03]] type='10000 * var * float32'>

In [9]:
Events.AnalysisElectrons.eta

<Array [[], [-0.861], ... [-1.21], [-0.238]] type='10000 * var * float32'>

The total in-memory size got a bit smaller due to the removed duplicated indices

In [10]:
Events.nbytes / (1024 ** 2)

322.89591217041016

In [11]:
ak.Array(array_dict).nbytes / (1024 ** 2)

404.93270111083984

# Write to parquet
Since `pyarrow>=2` we can write such a structure directly to parquet.

In [12]:
ak.to_parquet(Events, "physlite.parquet")

this gives us already in the default settings a fast readable format with about the same size that these branches originally took on disk. The reason why it is larger (or at least not smaller, despite deduplicated offsets) could be that the default compression in parquet is a bit less disk size efficient and more optimized for fast reading.

In [13]:
os.stat("physlite.parquet").st_size / (1024 ** 2)

112.27896118164062

In [14]:
sum([tree[k].compressed_bytes for k in array_dict]) / (1024 ** 2)

117.29965114593506

In [15]:
%%timeit
ak.from_parquet("physlite.parquet")

597 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can also be read this as a lazy array, such that branches are only read on demand.

In [16]:
%%time
lazy_parquet = ak.from_parquet("physlite.parquet", lazy=True)

CPU times: user 62.9 ms, sys: 12.4 ms, total: 75.3 ms
Wall time: 65.9 ms


In [17]:
lazy_parquet.AnalysisMuons.pt

<Array [[1.07e+05], [], ... [], [1.26e+05]] type='10000 * var * float32'>

Using `awkward` tag [1.1.0rc1](https://github.com/scikit-hep/awkward-1.0/releases/tag/1.1.0rc1) with `pyarrow>=3` this actually only caches the Muon pt:

In [18]:
lazy_parquet._caches

({'ak.from_parquet:0:off:AnalysisMuons.list.item.pt[0]': <ListOffsetArray64>
     <offsets><Index64 i="[0 1 1 1 1 ... 5302 5302 5302 5302 5303]" offset="0" length="10001" at="0x7f3a2b354540"/></offsets>
     <content><RecordArray>
         <field index="0" key="pt">
             <NumpyArray format="f" shape="5303" data="106870 25507.7 47097 74740.1 9129.62 ... 119823 60685.7 64930.9 37034.6 125925" at="0x7f3a2b34d800"/>
         </field>
     </RecordArray></content>
 </ListOffsetArray64>, 'ak.from_parquet:0:col:AnalysisMuons.list.item.pt[0]': <NumpyArray format="f" shape="5303" data="106870 25507.7 47097 74740.1 9129.62 ... 119823 60685.7 64930.9 37034.6 125925" at="0x7f3a23c7eec0"/>},)

# Write to HDF5 or other formats

One can store awkward arrays in basically any data format using `ak.to_buffers` which will separate the underlying 1-d arrays (content and indices for jagged arrays) and a json spec for the structure. See https://awkward-array.org/how-to-convert-buffers.html

In [19]:
form, length, container = ak.to_buffers(Events)

HDF5 is rather well suited for this since we can put the json form directly into the metadata.

In [20]:
import h5py

In [21]:
with h5py.File("physlite.h5", "w") as file:
    group = file.create_group("awkward")
    for k in container:
        v = container[k]
        group.create_dataset(k, shape=v.shape, dtype=v.dtype, data=v, compression="lzf")
    group.attrs["form"] = form.tojson()
    group.attrs["length"] = length

This file is quite a bit smaller. That makes sense since the compression can't compress away duplicated indices of different split branches (which we had before in the ROOT file).

In [22]:
os.stat("physlite.h5").st_size / (1024 ** 2)

105.9586591720581

Reading is also quite fast, although not as fast as with the parquet file (can be improved by using a faster compression, e.g. "lzf" at the cost of larger file size)

In [23]:
%%time
with h5py.File("physlite.h5", "r") as file:
    group = file["awkward"]
    reconstituted = ak.from_buffers(
        ak.forms.Form.fromjson(group.attrs["form"]),
        group.attrs["length"],
        {k: np.asarray(v) for k, v in group.items()},
    )

CPU times: user 885 ms, sys: 86.4 ms, total: 971 ms
Wall time: 971 ms


Reading only requested columns also works e.g. via "LazyArrays".

In [24]:
class LazyGet:
    def __init__(self, group):
        self.group = group
    
    def __getitem__(self, key):
        print(f"Reading array {key}")
        return np.asarray(self.group[key])

In [25]:
file = h5py.File("physlite.h5", "r")
group = file["awkward"]

lazy = ak.from_buffers(
    ak.forms.Form.fromjson(group.attrs["form"]),
    group.attrs["length"],
    LazyGet(group),
    lazy=True,
)

In [26]:
lazy.AnalysisElectrons.pt

Reading array part0-node917-offsets
Reading array part0-node923-data


<Array [[], [3.37e+03], ... [6.27e+03]] type='10000 * var * float32'>

In [27]:
lazy.AnalysisElectrons[ak.num(lazy.AnalysisElectrons.pt) > 0]

Reading array part0-node960-data
Reading array part0-node939-data


<Array [[{DFCommonElectronsECIDS: 0, ... ] type='4103 * var * {"DFCommonElectron...'>

One could also use very simple formats like `npz` (would need to store the metadata separately, e.g. in a json file)

In [28]:
np.savez_compressed("physlite.npz", **container)

In [29]:
os.stat("physlite.npz").st_size / (1024 ** 2)

77.33356857299805

In [30]:
%%time
with np.load("physlite.npz") as npf:
    reconstituted_np = ak.from_buffers(
        form,
        length,
        {k: v for k, v in npf.items()},
    )

CPU times: user 1.53 s, sys: 70.1 ms, total: 1.6 s
Wall time: 1.6 s


In [31]:
#file.close() # execute this to close the h5 file

# Behaviour and dynamic quantities
Working in a bit more object-oriented way can be done with "behaviours" (see https://awkward-array.readthedocs.io/en/latest/ak.behavior.html).

For example, coffea has LorentzVectors for awkward array.

In [32]:
from coffea.nanoevents.methods import vector
ak.behavior.update(vector.behavior)

The coffea `PtEtaPhiELorentzVector` calls the mass `mass`, but we call it `m`, so let's override that:

In [33]:
@ak.mixin_class(ak.behavior)
class xAODParticle(vector.PtEtaPhiELorentzVector):
    @property
    def mass(self):
        return self.m

Now, if name our Particles "xAODParticle" we can do all the LorentzVector stuff with them:

In [34]:
for collection in ["Electrons", "Jets", "Photons", "Muons"]:
    Events[f"Analysis{collection}"].layout.content.setparameter("__record__", "xAODParticle")

In [35]:
Events.AnalysisElectrons

<xAODParticleArray [[], ... firstEgMotherPdgId: -11}]] type='10000 * var * xAODP...'>

In [36]:
Events.AnalysisElectrons.delta_r(Events.AnalysisElectrons.nearest(Events.AnalysisJets))

<Array [[], [0.104], ... [0.0422], [0.943]] type='10000 * var * ?float32'>

Or something for track Particles:

In [37]:
@ak.mixin_class(ak.behavior)
class xAODTrackParticle(vector.LorentzVector):
    "see https://gitlab.cern.ch/atlas/athena/-/blob/21.2/Event/xAOD/xAODTracking/Root/TrackParticle_v1.cxx#L82"
    @property
    def theta(self):
        return self["theta"]
    
    @property
    def phi(self):
        return self["phi"]

    @property
    def p(self):
        return 1. / np.abs(self.qOverP)
    
    @property
    def x(self):
        return self.p * np.sin(self.theta) * np.cos(self.phi)
    
    @property
    def y(self):
        return self.p * np.sin(self.theta) * np.sin(self.phi)

    @property
    def z(self):
        return self.p * np.cos(self.theta)
    
    @property
    def t(self):
        return np.sqrt(139.570 ** 2 + sef.x ** 2 + self.y ** 2 + self.z ** 2)    

In [38]:
for k in Events.fields:
    if not "TrackParticles" in k:
        continue
    Events[k].layout.content.setparameter("__record__", "xAODTrackParticle")

In [39]:
Events.InDetTrackParticles

<xAODTrackParticleArray [[{phi: 2.36, ... ] type='10000 * var * xAODTrackParticl...'>

In [40]:
Events.InDetTrackParticles.pt

<Array [[1.11e+05, 1.7e+04, ... 5.57e+03, 739]] type='10000 * var * float32'>

# ElementLinks

Non-cyclic references can be implemented by just adding new indices and reusing the same contents. E.g let's link Electrons to their track particles:

In [41]:
def element_links(collection1, links, collection2):
    # Note: For proper handling one should read the
    # EventFormat.m_branchNames, EventFormat.m_branchHashes mapping to link to the correct collection
    # (possibly use UnionArray)
    # Also one could see if there is a better way to replicate the exact structure
    # instead of hardcoding
    return ak.Array(
        ak.layout.ListOffsetArray64(
            collection1.layout.offsets,
            ak.layout.ListArray64(
                links.layout.content.starts,
                links.layout.content.stops,
                ak.layout.IndexedArray64(
                    ak.layout.Index64(
                        ak.flatten(
                            links.m_persIndex
                            + ak.Array(np.array(collection2.layout.offsets[:-1])),
                            axis=None
                        )
                    ),
                    collection2.layout.content
                )
            )
        )
    )

In [42]:
Events["AnalysisElectrons", "trackParticles"] = element_links(
    Events.AnalysisElectrons,
    Events.AnalysisElectrons.trackParticleLinks,
    Events.GSFTrackParticles
)

In [43]:
Events.AnalysisElectrons.trackParticles

<xAODTrackParticleArray [[], ... chiSquared: 68}]]] type='10000 * var * var * xA...'>

In [44]:
# first track particle pt for each electron
Events.AnalysisElectrons.trackParticles[:,:,0].pt

<Array [[], [4.13e+03], ... [5.68e+03]] type='10000 * var * float32'>